O intervalo de confiança é dado pela fórmula:

$$Best\ Estimate \pm Margin\ of\ Error$$

Onde **Best Estimate** pode ser uma média ou uma proporção de uma amostra da população e a **Margin of Error** é dado por um **t** vezes o erro padrão **(SE)**. **t** depende do nível de confiança desejado.

O erro padrão (SE) é calculado direfente para proporção ou média.

Para proporção (p), o erro padrão (SE) é dado por:


$$SE_{proporção} = \sqrt{\frac{p\ (1 - p)}{n}}$$

Sendo $\sigma$ o Desvio Padrão, Para média o erro padrão (SE) é dado por:

$$SE_{média} = \frac{\sigma}{\sqrt{n}}$$

Encontrando o **t**. Para encontrar **t** para um nível de confiança de 90% significa que queremos um **t** no qual 90% da densidade da distribuição t está entre -**t** e **t**.


Mas como a distribuição t é simétrica, ao invés de acharmos:

P($-t^*$ < X < $t^*$) = .90

Podemo achar (da para ver desenhando tranquilamente): 

P(X < $t^*$) = .95

grau de liberdade é o tamanho da amostra menos o número de parâmetros.

Usando o pacote scipy.stats e definindo o grau de liberdade (df) como 10:

In [5]:
from scipy.stats import t
t.ppf(.95, df=10)

1.8124611228107335

In [33]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Intervalo de confiança em medidas de proporção para um grupo

Consideremos um amostra de tamanho 659 e com variável de interesse de proporção 85% (poderia ser de brasileiros que usam óculos). Podemos dizer que a um nível de 95% de confiança a proporção real da população (brasileiros) está entre entre 0,82 e 0,87.

Isto é, se selecionarmos outras amostras de mesmo tamanho muitas vezes, e criarmos outros intervalos de confiança, 95% desses intevalos conterão o proporção verdadeira (ou melhor, a proporção da população)

In [32]:
t = 1.96
p = .85
n = 659

se = np.sqrt((p * (1 - p))/n)

(p - t * se,p + t * se)

(0.8227373256215749, 0.8772626743784251)

In [5]:
# usando a biblioteca
sm.stats.proportion_confint(n * p, n)

(0.8227378265796143, 0.8772621734203857)

# Intervalo de confiança em medidas de média para um grupo

In [34]:
df = pd.read_csv("data/notas.csv")

media = df["matematica"].mean()
sd = df["matematica"].std()
n = len(df)

t = 2.064

se = sd/np.sqrt(n)
(media - t * se,media + t * se)

(6.001158177082324, 7.548841822917677)

In [35]:
# podemos usar a biblioteca stats:
sm.stats.DescrStatsW(df["matematica"]).zconfint_mean()

(6.040163709956654, 7.509836290043347)

## Intervalo de confiança em medidas proporção comparando-as entre dois grupos

A diferença entre duas proporções de amostra com base em dados independentes tem um erro padrão que reflete a incerteza combinada nas duas proporções

In [21]:
# vamos comparar o a proporção de fumantes entre dois grupos: homens e mulheres
# coluna SMQ020: 1 é fumante 2 não é fumante
# coluna RIAGENDR: 1 é homem e 2 mulheres
data = pd.read_csv('data/nhanes_2015_2016.csv')

In [22]:
homens = data[ data.RIAGENDR==1 ]
mulheres  = data[ data.RIAGENDR==2 ]

mulheres_fumantes = data[ (data.SMQ020 == 1) & (data.RIAGENDR==2) ]
homens_fumantes = data[ (data.SMQ020 == 1) & (data.RIAGENDR==1) ]

In [24]:
p_mulheres = len(mulheres_fumantes)/len(mulheres)
p_mulheres

0.30443548387096775

In [25]:
p_homens = len(homens_fumantes)/len(homens)
p_homens

0.5121420804639362

Como agora estamos diante de dois grupos e não apenas um, temos que ajustar o erro padrão:

$$SE = \sqrt{(SE_{\ 1})^2 + (SE_{\ 2})^2}$$

In [27]:
se_homens = np.sqrt( (p_homens * (1-p_homens))/(len(homens)) )
se_homens

0.009516254762527675

In [28]:
se_mulheres = np.sqrt( (p_mulheres * (1-p_mulheres))/(len(mulheres)) )
se_mulheres

0.008435287344906213

In [29]:
se = np.sqrt(se_homens**2 + se_mulheres**2)
se

0.012716649609722899

Considerando 95% de confiança (o mesmo que t igual à 1.96) podemos dizer que proporção de fumantes no grupo de homens e mulheres são diferentes, pois o intervalo de confiança dessa diferença (-0,23 até -0,18) não inclui zero.

In [31]:
diff = p_mulheres - p_homens
(diff - 1.96*se,diff + 1.96*se )

(-0.2326312298280253, -0.18278196335791153)

## Intervalo de confiança em medidas média comparando-as entre dois grupos

Vamos usar o coluna BMXBMI que é uma medida de indice de massa corporal (Body Mass Index). Os grupos que vamos comparar são homens e mulheres:

In [38]:
media_homens = np.mean(homens.BMXBMI)
media_homens

28.778072111846942

In [39]:
media_mulheres = np.mean(mulheres.BMXBMI)
media_mulheres

29.93994565217392

In [41]:
se_homens = np.std(homens.BMXBMI)/np.sqrt(len(homens))
se_homens

0.1190152572769987

In [42]:
se_mulheres = np.std(mulheres.BMXBMI)/np.sqrt(len(mulheres))
se_mulheres

0.14210108924105547

In [45]:
se = np.sqrt(se_homens**2 + se_mulheres**2)
se

0.18535736032918843

A um nível de 95% de confiança podemos dizer que há diferença entre o BMI médio de homens e mulheres pois o intervalo de confiança não inclui o zero: 

In [48]:
diff = media_mulheres - media_homens
(diff - 1.96*se,diff + 1.96*se )

(0.7985731140817667, 1.5251739665721853)

Considerações ou requisitos para aplicar esse técnica:

- A amostra deve ser representativa da população alvo, isto é, a amostra deve ser independete e identicamente distribuída na população
- Tamanho de amostra grande o suficiente. Grande o suficiente para podermos aproximar nossa distribuição amostral com uma curva normal. A medida que nosso N aumenta, nossa distribuição amostral se tornará cada vez mais mais em forma de sino. No caso de proporção n vezes p deve ser ao menos 10 e n * (1-p) também dever ser ao menos 10.

Erro padrão: nos diz o quão esperamos que uma estimativa fique próxima do valor verdadeiro